PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845493 sha256=7d9eda56540650a1a719a8b91ad49348ff9eeefa26ab473b2665acec5187b917
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Taipei Housing Price Prediction") \
    .getOrCreate()

# 타이베이 주택 가격 예측 모델 만들기




데이터셋 설명

이번 문제는 대만 타이베이 시의 신단 지역에서 수집된 주택 거래 관련 정보를 바탕으로 주택 가격(정확히는 주택의 평당 가격)을 예측하는 Regression 모델을 만들어보는 것이다. 총 6개의 피쳐와 주택의 평당 가격에 해당하는 레이블 정보가 훈련 데이터로 제공된다. 레이블의 경우에는 주택의 최종 가격이 아니라 평당 가격이란 점을 다시 한번 강조한다.

각 컬럼에 대한 설명은 아래와 같으며 모든 필드는 X4를 제외하고는 실수 타입이다.

* X1: 주택 거래 날짜를 실수로 제공한다. 소수점 부분은 달을 나타낸다. 예를 들어 2013.250이라면 2013년 3월임을 나타낸다 (0.250 = 3/12)
* X2: 주택 나이 (년수)
* X3: 가장 가까운 지하철역까지의 거리 (미터)
* X4: 주택 근방 걸어갈 수 있는 거리내 편의점 수
* X5: 주택 위치의 위도 (latitude)
* X6: 주택 위치의 경도 (longitude)
* Y: 주택 평당 가격



In [4]:
!wget https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv

--2024-07-11 11:00:48--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 3.5.141.22, 52.219.148.90, 3.5.145.122, ...
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|3.5.141.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20014 (20K) [text/csv]
Saving to: ‘Taipei_sindan_housing.csv’

Taipei_sindan_housi 100%[===================>]  19.54K  --.-KB/s    in 0s      

2024-07-11 11:00:48 (49.2 MB/s) - ‘Taipei_sindan_housing.csv’ saved [20014/20014]



In [5]:
!ls -tl

total 24
drwxr-xr-x 1 root root  4096 Jul  9 13:21 sample_data
-rw-r--r-- 1 root root 20014 Jul 17  2021 Taipei_sindan_housing.csv


In [6]:
data = spark.read.csv('./Taipei_sindan_housing.csv', header=True, inferSchema=True)

In [7]:
data.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: integer (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: double (nullable = true)
 |-- Y: double (nullable = true)



In [8]:
data.show()

+--------+----+--------+---+--------+---------+----+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|
+--------+----+--------+---+--------+---------+----+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|
|2013.083|34.8|405.2134|  1|24.97349|121.53372|41.4|
|2013.333| 6.3|90.45606|  9|24.97433| 121.5431|58.1|
|2012.917|13.0|492.2313|  5|24.96515|121.53737|39.3|
|2012.667|20.4|2469.645|  4|24.96108|121.51046|23.8|
|  2013.5|13.2|1164.838|  4|24.99156|121.53406|34.3|
|2013.583|35.7|579.2083|  2| 24.9824|121.54619

In [11]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[: -1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [12]:
feature_columns

['X1', 'X2', 'X3', 'X4', 'X5', 'X6']

In [13]:
data_2 = assembler.transform(data)

In [14]:
data_2.show()

+--------+----+--------+---+--------+---------+----+--------------------+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|            features|
+--------+----+--------+---+--------+---------+----+--------------------+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|[2012.917,32.0,84...|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|[2012.917,19.5,30...|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|[2013.583,13.3,56...|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|[2013.5,13.3,561....|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|[2012.833,5.0,390...|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|[2012.667,7.1,217...|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|[2012.667,34.5,62...|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|[2013.417,20.3,28...|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|[2013.5,31.7,5512...|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|[2013.417,17.9,17...|
|2013.083|34.8|405.2134|  1|24.97349|1

In [15]:
train, test = data_2.randomSplit([0.7, 0.3])

In [19]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="Y")
model = algo.fit(train)

In [20]:
evaluation_summary = model.evaluate(test)

In [21]:
evaluation_summary

In [22]:
evaluation_summary.meanAbsoluteError

6.0588492563433345

In [23]:
evaluation_summary.rootMeanSquaredError

7.970516205626599

In [24]:
evaluation_summary.r2

0.6570214994930936

In [25]:
predictions = model.transform(test)

In [27]:
predictions.show()

+--------+----+--------+---+--------+---------+----+--------------------+------------------+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|            features|        prediction|
+--------+----+--------+---+--------+---------+----+--------------------+------------------+
|2012.667| 1.5|23.38284|  7|24.96772|121.54102|47.7|[2012.667,1.5,23....|47.197552594509034|
|2012.667| 1.5|23.38284|  7|24.96772|121.54102|49.7|[2012.667,1.5,23....|47.197552594509034|
|2012.667|12.4|1712.632|  2|24.96412| 121.5167|31.3|[2012.667,12.4,17...| 31.11126681375754|
|2012.667|14.6|339.2289|  1|24.97519|121.53151|26.5|[2012.667,14.6,33...| 38.25961147041744|
|2012.667|23.0|130.9945|  6|24.95663|121.53765|37.2|[2012.667,23.0,13...|38.037428779975016|
|2012.667|29.1|506.1144|  4|24.97845|121.53889|40.3|[2012.667,29.1,50...| 38.13437845667795|
|2012.667|32.7|392.4459|  6|24.96398| 121.5425|30.5|[2012.667,32.7,39...| 36.48867076093666|
|2012.667|32.9|87.30222| 10|  24.983|121.54022|47.1|[2012.667,32.9,87.

In [26]:
predictions.select(predictions.columns[6: ]).show()

+----+--------------------+------------------+
|   Y|            features|        prediction|
+----+--------------------+------------------+
|47.7|[2012.667,1.5,23....|47.197552594509034|
|49.7|[2012.667,1.5,23....|47.197552594509034|
|31.3|[2012.667,12.4,17...| 31.11126681375754|
|26.5|[2012.667,14.6,33...| 38.25961147041744|
|37.2|[2012.667,23.0,13...|38.037428779975016|
|40.3|[2012.667,29.1,50...| 38.13437845667795|
|30.5|[2012.667,32.7,39...| 36.48867076093666|
|47.1|[2012.667,32.9,87...|46.214819073675244|
|40.3|[2012.667,34.5,62...| 39.59000545111303|
|44.0|[2012.75,0.0,208....| 43.31972837804642|
|43.2|[2012.75,3.2,489....|  46.8302324121687|
|46.8|[2012.75,11.4,390...| 44.38668249127295|
|34.1|[2012.75,12.5,114...| 42.68068289130133|
|55.1|[2012.75,15.4,205...| 47.39923129276758|
|37.5|[2012.75,15.6,752...|38.333527142087405|
|25.0|[2012.75,29.6,769...|41.215052000890864|
|21.5|[2012.75,31.4,144...|31.195711460586608|
|34.2|[2012.75,37.9,488...|32.175497290441854|
|55.2|[2012.8

In [28]:
model.save("Taipei_sindan_housing_model")

In [29]:
!ls -tl Taipei_sindan_housing_model

total 8
drwxr-xr-x 2 root root 4096 Jul 11 11:16 data
drwxr-xr-x 2 root root 4096 Jul 11 11:16 metadata


In [30]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [31]:
model_save_name = "Taipei_sindan_housing_model"
path = F"/content/gdrive/My Drive/Taipei_sindan_housing_model"

model.save(path)

In [32]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)

In [33]:
predictions2 = loaded_model.transform(test)

In [34]:
predictions2.select(predictions.columns[6: ]).show()

+----+--------------------+------------------+
|   Y|            features|        prediction|
+----+--------------------+------------------+
|47.7|[2012.667,1.5,23....|47.197552594509034|
|49.7|[2012.667,1.5,23....|47.197552594509034|
|31.3|[2012.667,12.4,17...| 31.11126681375754|
|26.5|[2012.667,14.6,33...| 38.25961147041744|
|37.2|[2012.667,23.0,13...|38.037428779975016|
|40.3|[2012.667,29.1,50...| 38.13437845667795|
|30.5|[2012.667,32.7,39...| 36.48867076093666|
|47.1|[2012.667,32.9,87...|46.214819073675244|
|40.3|[2012.667,34.5,62...| 39.59000545111303|
|44.0|[2012.75,0.0,208....| 43.31972837804642|
|43.2|[2012.75,3.2,489....|  46.8302324121687|
|46.8|[2012.75,11.4,390...| 44.38668249127295|
|34.1|[2012.75,12.5,114...| 42.68068289130133|
|55.1|[2012.75,15.4,205...| 47.39923129276758|
|37.5|[2012.75,15.6,752...|38.333527142087405|
|25.0|[2012.75,29.6,769...|41.215052000890864|
|21.5|[2012.75,31.4,144...|31.195711460586608|
|34.2|[2012.75,37.9,488...|32.175497290441854|
|55.2|[2012.8